##Factors
A factor is a function from an asset and a moment in time to a number.
```
F(asset, timestamp) -> float
```
In Pipeline, [Factors](https://www.quantopian.com/help#quantopian_pipeline_factors_Factor) are the most commonly-used term, representing the result of any computation producing a numerical result. Factors require a column of data and a window length as input.

The simplest factors in Pipeline are [built-in Factors](https://www.quantopian.com/help#built-in-factors). Built-in Factors are pre-built to perform common computations. As a first example, let's make a factor to compute the average close price over the last 10 days. We can use the `SimpleMovingAverage` built-in factor which computes the average value of the input data (close price) over the specified window length (10 days). To do this, we need to import our built-in `SimpleMovingAverage` factor and the [USEquityPricing dataset](https://www.quantopian.com/help#importing-datasets).

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# New from the last lesson, import the USEquityPricing dataset.
from quantopian.pipeline.data.builtin import USEquityPricing

# New from the last lesson, import the built-in SimpleMovingAverage factor.
from quantopian.pipeline.factors import SimpleMovingAverage

###Creating a Factor
Let's go back to our `make_pipeline` function from the previous lesson and instantiate a `SimpleMovingAverage` factor. To create a `SimpleMovingAverage` factor, we can call the `SimpleMovingAverage` constructor with two arguments: inputs, which must be a list of `BoundColumn` objects, and window_length, which must be an integer indicating how many days worth of data our moving average calculation should receive. (We'll discuss `BoundColumn` in more depth later; for now we just need to know that a `BoundColumn` is an object indicating what kind of data should be passed to our Factor.).

The following line creates a `Factor` for computing the 10-day mean close price of securities.

In [2]:
mean_close_15 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=15)

It's important to note that creating the factor does not actually perform a computation. Creating a factor is like defining the function. To perform a computation, we need to add the factor to our pipeline and run it.

###Adding a Factor to a Pipeline
Let's update our original empty pipeline to make it compute our new moving average factor. To start, let's move our factor instantatiation into `make_pipeline`. Next, we can tell our pipeline to compute our factor by passing it a `columns` argument, which should be a dictionary mapping column names to factors, filters, or classifiers. Our updated `make_pipeline` function should look something like this:

In [3]:
def make_pipeline():
    
    mean_close_15 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=15)
    
    return Pipeline(
        columns={
            '15_day_mean_close': mean_close_15
        }
    )

To see what this looks like, let's make our pipeline, run it, and display the result.

In [4]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-05')
result

15_day_mean_close
2019-05-06 00:00:00+00:00 Equity(2 [HWM])                  20.860296
                          Equity(21 [AAME])                 2.464000
                          Equity(24 [AAPL])               204.656667
                          Equity(25 [HWM_PR])              87.660000
                          Equity(41 [ARCB])                32.046000
                          Equity(52 [ABM])                 37.470667
                          Equity(53 [ABMD])               267.883000
                          Equity(62 [ABT])                 77.107333
                          Equity(64 [GOLD])                12.951667
                          Equity(66 [AB])                  29.124394
                          Equity(67 [ADSK])               173.966667
                          Equity(70 [VBF])                 18.334000
                          Equity(76 [TAP])                 61.398000
                          Equity(100 [IEP])                74.542333
                          Equity(106 [ACU])                20.485533
                          Equity(110 [RAMP])               57.510667
                          Equity(112 [ACY])                11.558778
                          Equity(114 [ADBE])              277.692667
                          Equity(117 [AEY])                 1.314800
                          Equity(122 [ADI])               114.939333
                          Equity(128 [ADM])                43.033000
                          Equity(149 [ADX])                15.049333
                          Equity(153 [AE])                 36.021000
                          Equity(154 [AEM])                41.130667
                          Equity(157 [AEG])                 5.163333
                          Equity(161 [AEP])                84.160000
                          Equity(166 [AES])                17.417789
                          Equity(185 [AFL])                49.461333
                          Equity(192 [ATAX])                6.908667
                          Equity(197 [AGCO])               71.898000
...                                                              ...
                          Equity(53097 [GNLN])             17.379500
                          Equity(53098 [SHLL])              9.721000
                          Equity(53099 [RYLD])             25.201556
                          Equity(53100 [SHLL_WS])           0.725000
                          Equity(53101 [MREO])              5.429286
                          Equity(53102 [SOHO_N])           25.406833
                          Equity(53103 [USI])              25.021500
                          Equity(53104 [ALTG_WS])           0.676667
                          Equity(53105 [SEIX])             25.034000
                          Equity(53106 [ALTG])              9.700000
                          Equity(53107 [ACTT_U])           10.083400
                          Equity(53108 [ETP_PRE])          24.857500
                          Equity(53109 [WTRU])             54.235500
                          Equity(53112 [WAFU])              4.610000
                          Equity(53113 [TPLC])             25.047500
                          Equity(53114 [TPHD])             24.957500
                          Equity(53115 [SY])               20.750000
                          Equity(53116 [BYND])             66.830000
                          Equity(53117 [TRVI])                   NaN
                          Equity(53118 [TMDX])             24.785000
                          Equity(53119 [KEY_PRK])                NaN
                          Equity(53120 [YJ])                     NaN
                          Equity(53121 [BUL])                    NaN
                          Equity(53122 [RRBI])                   NaN
                          Equity(53123 [SCPL])                   NaN
                          Equity(53124 [TRNE_WS])                NaN
                          Equity(53125 [TRNE])   

Now we have a column in our pipeline output with the 10-day average close price for all 8000+ securities (display truncated). Note that each row corresponds to the result of our computation for a given security on a given date stored. The `DataFrame` has a [MultiIndex](http://pandas.pydata.org/pandas-docs/version/0.16.2/advanced.html) where the first level is a datetime representing the date of the computation and the second level is an [Equity](http://localhost:3000/help#api-sidinfo) object corresponding to the security. For example, the first row (`2015-05-05 00:00:00+00:00`, `Equity(2 [AA])`) will contain the result of our `mean_close_10` factor for AA on May 5th, 2015.

If we run our pipeline over more than one day, the output looks like this.

In [5]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-07')
result

15_day_mean_close
2019-05-06 00:00:00+00:00 Equity(2 [HWM])                  20.860296
                          Equity(21 [AAME])                 2.464000
                          Equity(24 [AAPL])               204.656667
                          Equity(25 [HWM_PR])              87.660000
                          Equity(41 [ARCB])                32.046000
                          Equity(52 [ABM])                 37.470667
                          Equity(53 [ABMD])               267.883000
                          Equity(62 [ABT])                 77.107333
                          Equity(64 [GOLD])                12.951667
                          Equity(66 [AB])                  29.124394
                          Equity(67 [ADSK])               173.966667
                          Equity(70 [VBF])                 18.334000
                          Equity(76 [TAP])                 61.398000
                          Equity(100 [IEP])                74.542333
                          Equity(106 [ACU])                20.485533
                          Equity(110 [RAMP])               57.510667
                          Equity(112 [ACY])                11.558778
                          Equity(114 [ADBE])              277.692667
                          Equity(117 [AEY])                 1.314800
                          Equity(122 [ADI])               114.939333
                          Equity(128 [ADM])                43.033000
                          Equity(149 [ADX])                15.049333
                          Equity(153 [AE])                 36.021000
                          Equity(154 [AEM])                41.130667
                          Equity(157 [AEG])                 5.163333
                          Equity(161 [AEP])                84.160000
                          Equity(166 [AES])                17.417789
                          Equity(185 [AFL])                49.461333
                          Equity(192 [ATAX])                6.908667
                          Equity(197 [AGCO])               71.898000
...                                                              ...
2019-05-07 00:00:00+00:00 Equity(53099 [RYLD])             25.210400
                          Equity(53100 [SHLL_WS])           0.725000
                          Equity(53101 [MREO])              5.346250
                          Equity(53102 [SOHO_N])           25.404429
                          Equity(53103 [USI])              25.021500
                          Equity(53104 [ALTG_WS])           0.675000
                          Equity(53105 [SEIX])             25.034000
                          Equity(53106 [ALTG])              9.700000
                          Equity(53107 [ACTT_U])           10.084500
                          Equity(53108 [ETP_PRE])          24.848000
                          Equity(53109 [WTRU])             54.112400
                          Equity(53112 [WAFU])              4.545000
                          Equity(53113 [TPLC])             25.065000
                          Equity(53114 [TPHD])             24.953333
                          Equity(53115 [SY])               20.510000
                          Equity(53116 [BYND])             70.695000
                          Equity(53117 [TRVI])                   NaN
                          Equity(53118 [TMDX])             26.137500
                          Equity(53119 [KEY_PRK])          25.080000
                          Equity(53120 [YJ])               13.381000
                          Equity(53121 [BUL])              24.700000
                          Equity(53122 [RRBI])             49.390000
                          Equity(53123 [SCPL])             15.020000
                          Equity(53124 [TRNE_WS])                NaN
                          Equity(53125 [TRNE])              9.700000
                          Equity(53126 [PTIN])             24.960000
                          Equity(53127 [ATIF])   

Note: factors can also be added to an existing `Pipeline` instance using the `Pipeline.add` method. Using `add` looks something like this:
    >>> my_pipe = Pipeline()
    >>> f1 = SomeFactor(...)
    >>> my_pipe.add(f1, 'f1')

###Latest
The most commonly used built-in `Factor` is `Latest`. The `Latest` factor gets the most recent value of a given data column. This factor is common enough that it is instantiated differently from other factors. The best way to get the latest value of a data column is by getting its `.latest` attribute. As an example, let's update `make_pipeline` to create a latest close price factor and add it to our pipeline:

In [6]:
def make_pipeline():

    mean_close_15 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=15)
    latest_close = USEquityPricing.close.latest

    return Pipeline(
        columns={
            '15_day_mean_close': mean_close_15,
            'latest_close_price': latest_close
        }
    )

And now, when we make and run our pipeline again, there are two columns in our output dataframe. One column has the 10-day mean close price of each security, and the other has the latest close price.

In [7]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-05')
result.head(5)

15_day_mean_close  \
2019-05-06 00:00:00+00:00 Equity(2 [HWM])              20.860296   
                          Equity(21 [AAME])             2.464000   
                          Equity(24 [AAPL])           204.656667   
                          Equity(25 [HWM_PR])          87.660000   
                          Equity(41 [ARCB])            32.046000   

                                               latest_close_price  
2019-05-06 00:00:00+00:00 Equity(2 [HWM])                   22.40  
                          Equity(21 [AAME])                   NaN  
                          Equity(24 [AAPL])                211.78  
                          Equity(25 [HWM_PR])               88.00  
                          Equity(41 [ARCB])                 30.22

`.latest` can sometimes return things other than `Factors`. We'll see examples of other possible return types in later lessons.

## Default Inputs
Some factors have default inputs that should never be changed. For example the [VWAP built-in factor](https://www.quantopian.com/help#built-in-factors) is always calculated from `USEquityPricing.close` and `USEquityPricing.volume`. When a factor is always calculated from the same `BoundColumns`, we can call the constructor without specifying `inputs`.

In [8]:
from quantopian.pipeline.factors import VWAP
vwap = VWAP(window_length=15)

In the next lesson, we will look at combining factors.